In [2]:
!pip install ultralytics supervision -q
from ultralytics import YOLO
import cv2, pandas as pd, numpy as np
from pathlib import Path
import supervision as sv
import matplotlib.pyplot as plt
%matplotlib inline

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [11]:
#uploading using drive link instead using gdown
import gdown
gdown.download("https://drive.google.com/file/d/1WzQfRLui9kCuea5duw2ZhhlWhSwg5laQ/view?usp=drive_link",output='capsule.zip',quiet=False)



Downloading...
From: https://drive.google.com/file/d/1WzQfRLui9kCuea5duw2ZhhlWhSwg5laQ/view?usp=drive_link
To: /content/capsule.zip
109kB [00:00, 27.7MB/s]


'capsule.zip'

In [3]:
from google.colab import drive
drive.mount('/content/drive')
!unzip -q "/content/drive/MyDrive/capsule_yolo.zip" -d "/content/"


Mounted at /content/drive


In [4]:
import shutil
from pathlib import Path
import random
import os

# CONFIG
dataset_path = Path("capsule_yolo")
split_ratio = 0.8  # Move 80% of defects to train

# Define paths
train_imgs_dir = dataset_path / "images" / "train"
train_lbls_dir = dataset_path / "labels" / "train"
val_imgs_dir = dataset_path / "images" / "val"
val_lbls_dir = dataset_path / "labels" / "val"

print("Analyzing 'val' folder...")

# 1. Filter images
# In your previous script, good images were named "good_val_...".
# Defects were named "{defect_type}_...".
val_images = list(val_imgs_dir.glob("*.png"))
defect_images = [img for img in val_images if not img.name.startswith("good_")]

if not defect_images:
    print("No defect images found in val! Check your folder.")
else:
    # 2. Shuffle and Split
    random.seed(42) # For reproducibility
    random.shuffle(defect_images)

    count_to_move = int(len(defect_images) * split_ratio)
    files_to_move = defect_images[:count_to_move]

    print(f"Total defects found in val: {len(defect_images)}")
    print(f"Moving {count_to_move} defects to 'train'...")

    # 3. Move files and Fix Class ID
    for img_path in files_to_move:
        # Identify corresponding label file
        lbl_name = img_path.with_suffix(".txt").name
        src_lbl = val_lbls_dir / lbl_name

        dst_img = train_imgs_dir / img_path.name
        dst_lbl = train_lbls_dir / lbl_name

        # A. Move Image
        shutil.move(str(img_path), str(dst_img))

        # B. Move Label and Fix Class ID (1 -> 0)
        if src_lbl.exists():
            with open(src_lbl, 'r') as f:
                lines = f.readlines()

            fixed_lines = []
            for line in lines:
                parts = line.strip().split()
                if parts:
                    # Force the first number (Class ID) to be 0
                    parts[0] = "0"
                    fixed_lines.append(" ".join(parts))

            # Write the fixed content to the NEW location
            with open(dst_lbl, 'w') as f:
                f.write("\n".join(fixed_lines))

            # Delete the old label file
            src_lbl.unlink()

    print("Success! Files re-distributed.")
    print(f"Train contains: {len(list(train_imgs_dir.glob('*.png')))} images")
    print(f"Val contains:   {len(list(val_imgs_dir.glob('*.png')))} images")

Analyzing 'val' folder...
Total defects found in val: 109
Moving 87 defects to 'train'...
Success! Files re-distributed.
Train contains: 306 images
Val contains:   45 images


In [8]:
import yaml
import os

# Define the base directory for the unzipped dataset
dataset_base_dir = '/content/capsule_yolo'

# Define the content for data_capsule.yaml with absolute paths
data_yaml_content = {
    'train': os.path.join(dataset_base_dir, 'images', 'train'),
    'val': os.path.join(dataset_base_dir, 'images', 'val'),
    'nc': 1,  # Number of classes - corrected from 0 to 1
    'names': ['defect'] # Class names
}

# Define the path where the yaml file will be saved (in the current working directory)
yaml_file_path = 'data_capsule.yaml'

# Write the content to the yaml file
with open(yaml_file_path, 'w') as file:
    yaml.dump(data_yaml_content, file, default_flow_style=False)

print(f"Corrected '{yaml_file_path}' has been created successfully.")

# Display the content of the created YAML file
with open(yaml_file_path, 'r') as file:
    print(file.read())

Corrected 'data_capsule.yaml' has been created successfully.
names:
- defect
nc: 1
train: /content/capsule_yolo/images/train
val: /content/capsule_yolo/images/val



In [9]:

!yolo segment train \
    data=data_capsule.yaml \
    model=yolov8m-seg.pt \
    epochs=20 \
    imgsz=640 \
    batch=16 \
    patience=25 \
    project=capsule_training \
    name=best_run \
    exist_ok=True

print("ENTRAÎNEMENT TERMINÉ !")

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data_capsule.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=best_run, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=25, perspective=0.0, plots=True, pose=12.0,

In [ ]:
model = YOLO("capsule_runs/final_train/weights/best.pt")

# Test on one defective capsule
test_img = "capsule_yolo/images/val/crack_000.png"
results = model(test_img, save=True, conf=0.25)[0]

# Show result
plt.figure(figsize=(12,8))
plt.imshow(cv2.cvtColor(cv2.imread(results.path), cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("Perfect defect segmentation – your model works!")
plt.show()